In [3]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


import plotly.plotly as py
import plotly.graph_objs as go
import plotly

import igraph
from igraph import *
igraph.__version__

'0.7.1'

In [4]:
! python --version

Python 2.7.15


In [5]:
plotly.__version__

'2.7.0'

In [6]:
init_notebook_mode(connected=True)

In [7]:
# plotly.__version__

In [8]:
def get_layer(i):
    return int(math.floor(math.log(float(i) + 1, 2)))

def get_parent(i):
    return int(math.floor((i - 1) /2))

In [9]:
def get_parents(children, high,  no_root=True):
    print(children)
    start_id = []
    i = 1
    for c in children:
        if i == high:
            return []
        start_id.append(i)
        i += c + 1
    
    for i, x in enumerate(reversed(start_id)):
        if high > x:
            res = []
            c = x
            while x < high:
                res.append(x)
                x += 1
            return res
    return []

In [10]:
def create_sequence(*kwargs):
    return '<th> ' + ' '.join(['<cm> ' + str(n) + ' </cm>' for n in get_parents(*kwargs) + [i]]) + ' </th>'

In [24]:
def tree_plot(children=[2, 1, 4, 3], highlight = None, save=False):
#     G = Graph.Tree(nr_vertices, 2) # 2 stands for children number

    G = Graph()
    
#     root, article
    G.add_vertex(0)
    
#     first level comments
    base_vertix = len(children)
    vcount = 0
    
    for i, num in enumerate(children):
        vcount += 1
        G.add_vertex(vcount)
        G.add_edge(0, vcount)
        
        for ii in range(num):
            vcount += 1
            G.add_vertex(vcount)
            G.add_edge(vcount - 1, vcount)
    
    for x in [1, 4, 6, 9]:
        G.add_vertex(vcount + 1)
        G.add_edge(x, vcount + 1)
        vcount += 1
    
    nr_vertices = vcount + 1
    print(nr_vertices)
    
    v_label = map(str, range(nr_vertices))

    lay = G.layout('rt', root=[0])

    position = {k: lay[k] for k in range(nr_vertices)}
    Y = [lay[k][1] for k in range(nr_vertices)]
    M = max(Y)

    es = EdgeSeq(G) # sequence of edges
    E = [e.tuple for e in G.es] # list of edges

    L = len(position)
    Xn = [position[k][0] for k in range(L)]
    Yn = [2*M-position[k][1] for k in range(L)]
    Xe = []
    Ye = []
    for edge in E:
        Xe+=[position[edge[0]][0],position[edge[1]][0], None]
        Ye+=[2*M-position[edge[0]][1],2*M-position[edge[1]][1], None] 

    labels = v_label

    labels = ['article']
    
    bla = 0
    for l in v_label[1:]:
          
        labels.append(l)

    colors = ['#deebf7']
    if not highlight is None:
        parents_highlight = get_parents(children, highlight)

        for i in range(1, nr_vertices):
            if i in parents_highlight:
                colors.append('#fdcc8a')
            else:
                if i == highlight:
                    colors.append('#fc8d59')
                else:
                    colors.append('#9ecae1')
    else:
        colors += ['#9ecae1'] * (nr_vertices - 1)

    lines = go.Scatter(x=Xe,
                       y=Ye,
                       mode='lines',
                       line=dict(color='rgb(210,210,210)', width=1),
                       hoverinfo='none'
                       )
    dots = go.Scatter(x=Xn,
                      y=Yn,
                      mode='markers',
                      name='',
                      marker=dict(symbol='dot',
                                  size=100,
                                  color=colors,
                                  line=dict(color='rgb(10,10,10)', width=1)
                                  ),
                      text=labels,
                      hoverinfo='text',
                      opacity=1
                      )

    def make_annotations(pos, text, font_size=30, font_color='rgb(0,0,0)'):
        L=len((pos))
        if len((text))!=L:
            raise ValueError('The lists pos and text must have the same len')
        annotations = go.Annotations()
        for k in range(L):
            annotations.append(
                go.Annotation(
                    text=labels[k], # or replace labels with a different list for the text within the circle  
                    x=pos[k][0], y=2*M-position[k][1],
                    xref='x1', yref='y1',
                    font=dict(color=font_color, size=font_size),
                    showarrow=False)
            )
        return annotations  

    axis = dict(showline=False, # hide axis line, grid, ticklabels and  title
                zeroline=False,
                showgrid=False,
                showticklabels=False,
                )
    title = ''
    if not highlight is None:
        title=  create_sequence(children, highlight)
    layout = dict(title=title,  
                  annotations=make_annotations(position, v_label),
                  font=dict(size=20),
                  showlegend=False,
                  xaxis=go.XAxis(axis),
                  yaxis=go.YAxis(axis),          
                  margin=dict(l=40, r=40, b=85, t=100),
                  hovermode='closest',
#                   plot_bgcolor='rgb(248,248,248)'          
                  )

    data=go.Data([lines, dots])
    fig=dict(data=data, layout=layout)
    fig['layout'].update(annotations=make_annotations(position, v_label))
    if save:
        iplot(fig, filename=str(highlight) , image='png', image_width=1000, image_height=1000)
    else:
        iplot(fig, image_width=1000, image_height=1000)

    return fig

In [25]:
for i in range(10, 11):
    fig = tree_plot(highlight=i, save=True)

19
[2, 1, 4, 3]
[2, 1, 4, 3]


In [13]:
tree_plot(save=True)

15


{'data': [{'hoverinfo': 'none',
   'line': {'color': 'rgb(210,210,210)', 'width': 1},
   'mode': 'lines',
   'type': 'scatter',
   'x': [0.0,
    -1.5,
    None,
    -1.5,
    -1.5,
    None,
    -1.5,
    -1.5,
    None,
    0.0,
    -0.5,
    None,
    -0.5,
    -0.5,
    None,
    0.0,
    0.5,
    None,
    0.5,
    0.5,
    None,
    0.5,
    0.5,
    None,
    0.5,
    0.5,
    None,
    0.5,
    0.5,
    None,
    0.0,
    1.5,
    None,
    1.5,
    1.5,
    None,
    1.5,
    1.5,
    None,
    1.5,
    1.5,
    None],
   'y': [10.0,
    9.0,
    None,
    9.0,
    8.0,
    None,
    8.0,
    7.0,
    None,
    10.0,
    9.0,
    None,
    9.0,
    8.0,
    None,
    10.0,
    9.0,
    None,
    9.0,
    8.0,
    None,
    8.0,
    7.0,
    None,
    7.0,
    6.0,
    None,
    6.0,
    5.0,
    None,
    10.0,
    9.0,
    None,
    9.0,
    8.0,
    None,
    8.0,
    7.0,
    None,
    7.0,
    6.0,
    None]},
  {'hoverinfo': 'text',
   'marker': {'color': ['#deebf7',
    

In [14]:
fig

{'data': [{'hoverinfo': 'none',
   'line': {'color': 'rgb(210,210,210)', 'width': 1},
   'mode': 'lines',
   'type': 'scatter',
   'x': [0.0,
    -1.5,
    None,
    -1.5,
    -1.5,
    None,
    -1.5,
    -1.5,
    None,
    0.0,
    -0.5,
    None,
    -0.5,
    -0.5,
    None,
    0.0,
    0.5,
    None,
    0.5,
    0.5,
    None,
    0.5,
    0.5,
    None,
    0.5,
    0.5,
    None,
    0.5,
    0.5,
    None,
    0.0,
    1.5,
    None,
    1.5,
    1.5,
    None,
    1.5,
    1.5,
    None,
    1.5,
    1.5,
    None],
   'y': [10.0,
    9.0,
    None,
    9.0,
    8.0,
    None,
    8.0,
    7.0,
    None,
    10.0,
    9.0,
    None,
    9.0,
    8.0,
    None,
    10.0,
    9.0,
    None,
    9.0,
    8.0,
    None,
    8.0,
    7.0,
    None,
    7.0,
    6.0,
    None,
    6.0,
    5.0,
    None,
    10.0,
    9.0,
    None,
    9.0,
    8.0,
    None,
    8.0,
    7.0,
    None,
    7.0,
    6.0,
    None]},
  {'hoverinfo': 'text',
   'marker': {'color': ['#deebf7',
    

In [146]:
iplot(fig, filename='test', image='png')

In [121]:
plot(fig, image = 'jpg', image_filename='plot_image', output_type='file', image_width=800, image_height=600)

TypeError: not all arguments converted during string formatting

In [30]:
def tree_plot_ed(children=[2, 1, 3], highlight = None, save=False):
#     G = Graph.Tree(nr_vertices, 2) # 2 stands for children number

    G = Graph()
    
#     root, article
    G.add_vertex(0)
    
#     first level comments
    base_vertix = len(children)
    vcount = 0
    
    for i, num in enumerate(children):
        vcount += 1
        G.add_vertex(vcount)
        G.add_edge(0, vcount)
        
        for ii in range(num):
            vcount += 1
            G.add_vertex(vcount)
            G.add_edge(vcount - 1, vcount)
    
    nr_vertices = vcount + 1
    print(nr_vertices)
    
    v_label = map(str, range(nr_vertices))

    lay = G.layout('rt', root=[0])

    position = {k: lay[k] for k in range(nr_vertices)}
    Y = [lay[k][1] for k in range(nr_vertices)]
    M = max(Y)

    es = EdgeSeq(G) # sequence of edges
    E = [e.tuple for e in G.es] # list of edges

    L = len(position)
    Xn = [position[k][0] for k in range(L)]
    Yn = [2*M-position[k][1] for k in range(L)]
    Xe = []
    Ye = []
    for edge in E:
        Xe+=[position[edge[0]][0],position[edge[1]][0], None]
        Ye+=[2*M-position[edge[0]][1],2*M-position[edge[1]][1], None] 

    labels = v_label

    labels = ['article']
    
    bla = 0
    for l in v_label[1:]:
          
        labels.append(l)

    colors = ['#deebf7']
    if True:
        parents_highlight = get_parents(children, highlight)

        for i in range(1, nr_vertices):
            if i in parents_highlight:
                colors.append('#fdcc8a')
            else:
                if i in [1, 4, 6]:
                    colors.append('#fdcc8a')
                else:
                    colors.append('#9ecae1')
    else:
        colors += ['#9ecae1'] * (nr_vertices - 1)

    lines = go.Scatter(x=Xe,
                       y=Ye,
                       mode='lines',
                       line=dict(color='rgb(210,210,210)', width=1),
                       hoverinfo='none'
                       )
    dots = go.Scatter(x=Xn,
                      y=Yn,
                      mode='markers',
                      name='',
                      marker=dict(symbol='dot',
                                  size=100,
                                  color=colors,
                                  line=dict(color='rgb(10,10,10)', width=1)
                                  ),
                      text=labels,
                      hoverinfo='text',
                      opacity=1
                      )

    def make_annotations(pos, text, font_size=30, font_color='rgb(0,0,0)'):
        L=len((pos))
        if len((text))!=L:
            raise ValueError('The lists pos and text must have the same len')
        annotations = go.Annotations()
        for k in range(L):
            annotations.append(
                go.Annotation(
                    text=labels[k], # or replace labels with a different list for the text within the circle  
                    x=pos[k][0], y=2*M-position[k][1],
                    xref='x1', yref='y1',
                    font=dict(color=font_color, size=font_size),
                    showarrow=False)
            )
        return annotations  

    axis = dict(showline=False, # hide axis line, grid, ticklabels and  title
                zeroline=False,
                showgrid=False,
                showticklabels=False,
                )
    title = ''
    if not highlight is None:
        title=  create_sequence(children, highlight)
    layout = dict(annotations=make_annotations(position, v_label),
                  font=dict(size=20),
                  showlegend=False,
                  xaxis=go.XAxis(axis),
                  yaxis=go.YAxis(axis),          
#                   margin=dict(l=40, r=40, b=85, t=100),
                  hovermode='closest',
#                   plot_bgcolor='rgb(248,248,248)'          
                  )

    data=go.Data([lines, dots])
    fig=dict(data=data, layout=layout)
    fig['layout'].update(annotations=make_annotations(position, v_label))
    if save:
        iplot(fig, filename=str(highlight) , image='png', image_width=650, image_height=1000)
    else:
        iplot(fig, image_width=500, image_height=1000)

    return fig

In [31]:
tree_plot_ed(save=True)

10
[2, 1, 3]


{'data': [{'hoverinfo': 'none',
   'line': {'color': 'rgb(210,210,210)', 'width': 1},
   'mode': 'lines',
   'type': 'scatter',
   'x': [0.0,
    -1.0,
    None,
    -1.0,
    -1.0,
    None,
    -1.0,
    -1.0,
    None,
    0.0,
    0.0,
    None,
    0.0,
    0.0,
    None,
    0.0,
    1.0,
    None,
    1.0,
    1.0,
    None,
    1.0,
    1.0,
    None,
    1.0,
    1.0,
    None],
   'y': [8.0,
    7.0,
    None,
    7.0,
    6.0,
    None,
    6.0,
    5.0,
    None,
    8.0,
    7.0,
    None,
    7.0,
    6.0,
    None,
    8.0,
    7.0,
    None,
    7.0,
    6.0,
    None,
    6.0,
    5.0,
    None,
    5.0,
    4.0,
    None]},
  {'hoverinfo': 'text',
   'marker': {'color': ['#deebf7',
     '#fdcc8a',
     '#9ecae1',
     '#9ecae1',
     '#fdcc8a',
     '#9ecae1',
     '#fdcc8a',
     '#9ecae1',
     '#9ecae1',
     '#9ecae1'],
    'line': {'color': 'rgb(10,10,10)', 'width': 1},
    'size': 100,
    'symbol': 'dot'},
   'mode': 'markers',
   'name': '',
   'opacity': 1,
 